# Load data 

In [2]:
import torch
import torchvision 
import torchvision.transforms as transoforms
import torch.optim as optim


from documents.load_data import *


train_loader, test_loader,val_loader, test_loaders = loader()(subject="ALL",
                                                 batch_size= 20,
                                                 num_validation = 37)

Train data shape:  (1782, 22, 1000)
Train labels shape:  (1782,)
test data shape:  (443, 22, 1000)
test labels shape:  (443,)
Validation data shape:  (333, 22, 1000)
Validation labels shape:  (333,)


NameError: name 'train_loader' is not defined